# Julia Basics

In this section, we are going to run julia with Jupyter    

## Activate Environment

In [1]:
using Pkg
Pkg.activate("tutorial")
Pkg.instantiate()

  Activating project at `~/work/Julia-for-SciML/Julia-for-SciML/hands-on/tutorial`
    Updating registry at `~/.julia/registries/General.toml`
   Installed OptimizationFlux ─────────── v0.1.1
   Installed OptimizationPolyalgorithms ─ v0.1.1
   Installed DifferentialEquations ────── v7.6.0
   Installed Sundials ─────────────────── v4.10.3
   Installed DelayDiffEq ──────────────── v5.38.2
   Installed DocStringExtensions ──────── v0.9.2
   Installed StochasticDiffEq ─────────── v6.55.0
   Installed SciMLSensitivity ─────────── v7.10.2
   Installed SparseDiffTools ──────────── v1.28.0
   Installed ExponentialUtilities ─────── v1.20.0
    Updating `~/work/Julia-for-SciML/Julia-for-SciML/hands-on/tutorial/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.1
  [2445eb08] + DataDrivenDiffEq v0.8.5
  [aae7a2af] + DiffEqFlux v1.52.0
  [41bf760c] + DiffEqSensitivity v6.79.0
  [0c46a032] + DifferentialEquations v7.6.0
  [587475ba] + Flux v0.13.6
  [961ee093] + ModelingToolkit v8.29.1
  [429524aa] +

In [2]:
] st

Status `~/work/Julia-for-SciML/Julia-for-SciML/hands-on/tutorial/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.1
  [2445eb08] DataDrivenDiffEq v0.8.5
  [aae7a2af] DiffEqFlux v1.52.0
  [41bf760c] DiffEqSensitivity v6.79.0
  [0c46a032] DifferentialEquations v7.6.0
  [587475ba] Flux v0.13.6
  [961ee093] ModelingToolkit v8.29.1
  [429524aa] Optim v1.7.3
  [7f7a1694] Optimization v3.9.2
  [91a5bcdd] Plots v1.35.5


## Import packages

In [3]:
using BenchmarkTools

In [4]:
using DifferentialEquations

┌ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1664
┌ Warning: Module ForwardDiff with build ID 138120893810 is missing from the cache.
│ This may mean ForwardDiff [f6369f11-7733-5829-9624-2563aa707210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1325
┌ Warning: Module ForwardDiff with build ID 138120893810 is missing from the cache.
│ This may mean ForwardDiff [f6369f11-7733-5829-9624-2563aa707210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1325
┌ Warning: Module ForwardDiff with build ID 138120893810 is missing from the cache.
│ This may mean ForwardDiff [f6369f11-7733-5829-9624-2563aa707210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1325
┌ Warning: Module ForwardDiff with build ID 138120893810 is missing from the cache.
│ This may mean ForwardDiff [f6369f11-7733-5829-9624-2563aa707210] d

## Documentation

In [5]:
?ODEProblem

search: ODEProblem RODEProblem SplitODEProblem DynamicalODEProblem



Defines an ordinary differential equation (ODE) problem. Documentation Page: https://diffeq.sciml.ai/stable/types/ode_types/

## Mathematical Specification of an ODE Problem

To define an ODE Problem, you simply need to give the function $f$ and the initial condition $u_0$ which define an ODE:

$$
M \frac{du}{dt} = f(u,p,t)
$$

There are two different ways of specifying `f`:

  * `f(du,u,p,t)`: in-place. Memory-efficient when avoiding allocations. Best option for most cases unless mutation is not allowed.
  * `f(u,p,t)`: returning `du`. Less memory-efficient way, particularly suitable when mutation is not allowed (e.g. with certain automatic differentiation packages such as Zygote).

`u₀` should be an AbstractArray (or number) whose geometry matches the desired geometry of `u`. Note that we are not limited to numbers or vectors for `u₀`; one is allowed to provide `u₀` as arbitrary matrices / higher dimension tensors as well.

For the mass matrix $M$, see the documentation of `ODEFunction`.

## Problem Type

### Constructors

`ODEProblem` can be constructed by first building an `ODEFunction` or by simply passing the ODE right-hand side to the constructor. The constructors are:

  * `ODEProblem(f::ODEFunction,u0,tspan,p=NullParameters();kwargs...)`
  * `ODEProblem{isinplace,specialize}(f,u0,tspan,p=NullParameters();kwargs...)` : Defines the ODE with the specified functions. `isinplace` optionally sets whether the function is inplace or not. This is determined automatically, but not inferred. `specialize` optionally controls the specialization level. See the [specialization levels section of the SciMLBase documentation](https://scimlbase.sciml.ai/stable/interfaces/Problems/#Specialization-Levels) for more details. The default is `AutoSpecialize`.

For more details on the in-place and specialization controls, see the ODEFunction documentation.

Parameters are optional, and if not given then a `NullParameters()` singleton will be used which will throw nice errors if you try to index non-existent parameters. Any extra keyword arguments are passed on to the solvers. For example, if you set a `callback` in the problem, then that `callback` will be added in every solve call.

For specifying Jacobians and mass matrices, see the `ODEFunction` documentation.

### Fields

  * `f`: The function in the ODE.
  * `u0`: The initial condition.
  * `tspan`: The timespan for the problem.
  * `p`: The parameters.
  * `kwargs`: The keyword arguments passed onto the solves.

## Example Problem

```julia
using SciMLBase
function lorenz!(du,u,p,t)
 du[1] = 10.0(u[2]-u[1])
 du[2] = u[1]*(28.0-u[3]) - u[2]
 du[3] = u[1]*u[2] - (8/3)*u[3]
end
u0 = [1.0;0.0;0.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz!,u0,tspan)

# Test that it worked
using OrdinaryDiffEq
sol = solve(prob,Tsit5())
using Plots; plot(sol,vars=(1,2,3))
```

## More Example Problems

Example problems can be found in [DiffEqProblemLibrary.jl](https://github.com/SciML/DiffEqProblemLibrary.jl).

To use a sample problem, such as `prob_ode_linear`, you can do something like:

```julia
#] add ODEProblemLibrary
using ODEProblemLibrary
prob = ODEProblemLibrary.prob_ode_linear
sol = solve(prob)
```

---

```
ODEProblem(f::ODEFunction,u0,tspan,p=NullParameters(),callback=CallbackSet())
```

Define an ODE problem from an [`ODEFunction`](@ref).
